In [40]:
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import geometry
from pyproj import Transformer
import py3dep

data_dir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar')
ncs_dir = data_dir.joinpath('ncs')
tree_dir = data_dir.joinpath('trees')
nlcd = tree_dir.joinpath('nlcd')

In [37]:
ds = xr.open_dataset(Path('/bsuhome/zacharykeskinen/scratch/spicy-lowman-quadrant/combo_v3.nc'))
ds = ds.reindex(y=list(reversed(ds.y)))
# get tree percentage 
print('tree percentage')

tree_perc = xr.open_dataarray(nlcd.joinpath('nlcd_2016_treecanopy_2019_08_31.img')).squeeze('band', drop =  True)

# find bounds in this datasets crs to clip it before reprojecting
# https://pyproj4.github.io/pyproj/stable/api/transformer.html
transformer = Transformer.from_crs("epsg:4326","epsg:5070", always_xy = True)
x_min, y_min, x_max, y_max = ds.rio.bounds()
ds_bds = [x_min - 0.5, y_min - 0.6, x_max + 0.6, y_max + 0.6]
bds = list(transformer.transform(*ds_bds[:2]))
bds.extend(list(transformer.transform(*ds_bds[2:])))
# clip big raster to our area
tree_perc = tree_perc.rio.clip_box(*bds)
# add our crs to reproject
ds = ds.rio.write_crs('epsg:4326')
# reproject and mask out areas where the interpolation led to artifacts
tree_perc = tree_perc.rio.reproject_match(ds['snow_depth'])
tree_perc = tree_perc.where((tree_perc >= 0) & (tree_perc < 100)) # percentage bounds 0-100
# add to dataset
ds['tree_perc'] = tree_perc

tree percentage


In [ ]:
dem = py3dep.get_map("DEM", geometry.box(*ds.rio.bounds()), resolution=30, geo_crs="epsg:4326", crs="epsg:4326")
ds['dem'] = dem.interp_like(ds['snow_depth'])

In [47]:
ds[['tree_perc','dem']].to_netcdf('/bsuhome/zacharykeskinen/scratch/spicy-lowman-quadrant/tree_dem.nc')